# Cryptocurrency Data Downloader
## Download Latest Data (2020-2025) for Multiple Cryptocurrencies

In [14]:
# Install required libraries (run once)
# !pip install yfinance pandas numpy requests

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import time
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d')}")

Libraries imported successfully!
Current date: 2025-11-14


## 1. Define Cryptocurrencies to Download

In [15]:
# List of cryptocurrencies (Add/Remove as needed)
crypto_symbols = [
    'BTC-USD',   # Bitcoin
    'ETH-USD',   # Ethereum
    'ADA-USD',   # Cardano
    'BNB-USD',   # Binance Coin
    'XRP-USD',   # Ripple
    'SOL-USD',   # Solana
    'DOT-USD',   # Polkadot
    'DOGE-USD',  # Dogecoin
    'MATIC-USD', # Polygon
    'AVAX-USD',  # Avalanche
    'LINK-USD',  # Chainlink
    'UNI-USD',   # Uniswap
    'LTC-USD',   # Litecoin
    'ATOM-USD',  # Cosmos
    'XLM-USD',   # Stellar
    'ALGO-USD',  # Algorand
    'VET-USD',   # VeChain
    'TRX-USD',   # Tron
    'FIL-USD',   # Filecoin
    'ETC-USD',   # Ethereum Classic
    'XMR-USD',   # Monero
    'AAVE-USD',  # Aave
    'THETA-USD', # Theta
    'EOS-USD',   # EOS
    'XTZ-USD',   # Tezos
]

# Create mapping of symbols to clean names
crypto_names = {
    'BTC-USD': 'Bitcoin',
    'ETH-USD': 'Ethereum',
    'ADA-USD': 'Cardano',
    'BNB-USD': 'Binance Coin',
    'XRP-USD': 'Ripple',
    'SOL-USD': 'Solana',
    'DOT-USD': 'Polkadot',
    'DOGE-USD': 'Dogecoin',
    'MATIC-USD': 'Polygon',
    'AVAX-USD': 'Avalanche',
    'LINK-USD': 'Chainlink',
    'UNI-USD': 'Uniswap',
    'LTC-USD': 'Litecoin',
    'ATOM-USD': 'Cosmos',
    'XLM-USD': 'Stellar',
    'ALGO-USD': 'Algorand',
    'VET-USD': 'VeChain',
    'TRX-USD': 'Tron',
    'FIL-USD': 'Filecoin',
    'ETC-USD': 'Ethereum Classic',
    'XMR-USD': 'Monero',
    'AAVE-USD': 'Aave',
    'THETA-USD': 'Theta',
    'EOS-USD': 'EOS',
    'XTZ-USD': 'Tezos',
}

print(f"Total cryptocurrencies to download: {len(crypto_symbols)}")
print("\nList:")
for i, symbol in enumerate(crypto_symbols, 1):
    print(f"{i:2d}. {crypto_names.get(symbol, symbol)}")

Total cryptocurrencies to download: 25

List:
 1. Bitcoin
 2. Ethereum
 3. Cardano
 4. Binance Coin
 5. Ripple
 6. Solana
 7. Polkadot
 8. Dogecoin
 9. Polygon
10. Avalanche
11. Chainlink
12. Uniswap
13. Litecoin
14. Cosmos
15. Stellar
16. Algorand
17. VeChain
18. Tron
19. Filecoin
20. Ethereum Classic
21. Monero
22. Aave
23. Theta
24. EOS
25. Tezos


## 2. Set Date Range

In [16]:
# Define date range
START_DATE = '2020-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')  # Today's date

print(f"Downloading data from {START_DATE} to {END_DATE}")
print(f"Total duration: ~{(datetime.now() - datetime.strptime(START_DATE, '%Y-%m-%d')).days} days")

Total duration: ~2144 days


## 3. Download Data Function

In [17]:
def download_crypto_data(symbol, start_date, end_date):
    """
    Download cryptocurrency data from Yahoo Finance
    
    Parameters:
    - symbol: Crypto symbol (e.g., 'BTC-USD')
    - start_date: Start date (YYYY-MM-DD)
    - end_date: End date (YYYY-MM-DD)
    
    Returns:
    - DataFrame with OHLCV data
    """
    try:
        # Download data
        data = yf.download(symbol, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print(f"  ❌ No data found for {symbol}")
            return None
        
        # Reset index to make Date a column
        data = data.reset_index()
        
        # Rename columns to match your format
        data = data.rename(columns={
            'Date': 'Date',
            'Open': 'Open',
            'High': 'High',
            'Low': 'Low',
            'Close': 'Close',
            'Volume': 'Volume',
            'Adj Close': 'Adj_Close'
        })
        
        # Add metadata columns
        clean_symbol = symbol.replace('-USD', '')
        data['Symbol'] = clean_symbol
        data['Name'] = crypto_names.get(symbol, clean_symbol)
        
        # Calculate Market Cap (approximation: Close * Volume)
        # Note: This is not real market cap, just a placeholder
        data['Marketcap'] = data['Close'] * data['Volume']
        
        # Add SNo
        data.insert(0, 'SNo', range(1, len(data) + 1))
        
        # Select and reorder columns to match your format
        columns_order = ['SNo', 'Name', 'Symbol', 'Date', 'High', 'Low', 
                        'Open', 'Close', 'Volume', 'Marketcap']
        data = data[columns_order]
        
        return data
        
    except Exception as e:
        print(f"  ❌ Error downloading {symbol}: {str(e)}")
        return None

print("Download function ready!")

Download function ready!


## 4. Download All Cryptocurrencies

In [18]:
# Create directories
os.makedirs('crypto_data_new', exist_ok=True)
os.makedirs('crypto_data_old', exist_ok=True)  # Backup your old data here

print("Starting download...\n")
print("=" * 70)

downloaded_data = {}
successful_downloads = 0
failed_downloads = 0

for i, symbol in enumerate(crypto_symbols, 1):
    print(f"[{i}/{len(crypto_symbols)}] Downloading {crypto_names.get(symbol, symbol)} ({symbol})...")
    
    # Download data
    df = download_crypto_data(symbol, START_DATE, END_DATE)
    
    if df is not None:
        # Save to CSV
        clean_symbol = symbol.replace('-USD', '')
        filename = f'crypto_data_new/{clean_symbol.lower()}_data.csv'
        df.to_csv(filename, index=False)
        
        downloaded_data[symbol] = df
        successful_downloads += 1
        
        print(f"  ✅ Downloaded {len(df)} records")
        print(f"  💾 Saved to: {filename}")
    else:
        failed_downloads += 1
    
    print()
    
    # Small delay to avoid rate limiting
    time.sleep(0.5)

print("=" * 70)
print(f"\n✅ Successfully downloaded: {successful_downloads}/{len(crypto_symbols)}")
print(f"❌ Failed downloads: {failed_downloads}/{len(crypto_symbols)}")

Starting download...

[1/25] Downloading Bitcoin (BTC-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/btc_data.csv

[2/25] Downloading Ethereum (ETH-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/eth_data.csv

[3/25] Downloading Cardano (ADA-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/ada_data.csv

[4/25] Downloading Binance Coin (BNB-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/bnb_data.csv

[5/25] Downloading Ripple (XRP-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/xrp_data.csv

[6/25] Downloading Solana (SOL-USD)...
  ✅ Downloaded 2044 records
  💾 Saved to: crypto_data_new/sol_data.csv

[7/25] Downloading Polkadot (DOT-USD)...
  ✅ Downloaded 1912 records
  💾 Saved to: crypto_data_new/dot_data.csv

[8/25] Downloading Dogecoin (DOGE-USD)...
  ✅ Downloaded 2144 records
  💾 Saved to: crypto_data_new/doge_data.csv

[9/25] Downloading Polygon (MATIC-USD)...
  ✅ Downloaded 1910 records
  💾 

## 5. Combine All Data into Single File

In [19]:
# Combine all downloaded data
if downloaded_data:
    combined_df = pd.concat(downloaded_data.values(), ignore_index=True)
    
    # Sort by Symbol and Date
    combined_df = combined_df.sort_values(['Symbol', 'Date']).reset_index(drop=True)
    
    # Update SNo
    combined_df['SNo'] = range(1, len(combined_df) + 1)
    
    # Save combined file
    combined_df.to_csv('crypto_data_combined.csv', index=False)
    
    print("\n" + "=" * 70)
    print("COMBINED DATASET SUMMARY")
    print("=" * 70)
    print(f"Total records: {len(combined_df):,}")
    print(f"Cryptocurrencies: {combined_df['Symbol'].nunique()}")
    print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"File saved: crypto_data_combined.csv")
    print("=" * 70)
    
    # Show sample
    print("\nSample data:")
    display(combined_df.head(10))
    
    # Summary by coin
    # Add this before line 29:
    print(combined_df.columns.tolist())
    print("\nRecords per cryptocurrency:")
    print(combined_df.head())
else:
    print("No data was downloaded successfully.")


COMBINED DATASET SUMMARY
Total records: 52,286
Cryptocurrencies: 25
Date range: 2020-01-01 00:00:00 to 2025-11-13 00:00:00
File saved: crypto_data_combined.csv

Sample data:


Price,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,...,High,Low,Open,Close,Volume,High,Low,Open,Close,Volume
Ticker,,,,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,,...,EOS-USD,EOS-USD,EOS-USD,EOS-USD,EOS-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD
0,1,Aave,AAVE,2020-10-02,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Aave,AAVE,2020-10-03,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Aave,AAVE,2020-10-04,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Aave,AAVE,2020-10-05,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Aave,AAVE,2020-10-06,NaN,NaN,NaN,NaN,NaN,2.472399e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Aave,AAVE,2020-10-07,NaN,NaN,NaN,NaN,NaN,2.737070e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Aave,AAVE,2020-10-08,NaN,NaN,NaN,NaN,NaN,7.259723e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Aave,AAVE,2020-10-09,NaN,NaN,NaN,NaN,NaN,3.818165e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Aave,AAVE,2020-10-10,NaN,NaN,NaN,NaN,NaN,5.280042e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[('SNo', ''), ('Name', ''), ('Symbol', ''), ('Date', ''), ('High', 'BTC-USD'), ('Low', 'BTC-USD'), ('Open', 'BTC-USD'), ('Close', 'BTC-USD'), ('Volume', 'BTC-USD'), ('Marketcap', ''), ('High', 'ETH-USD'), ('Low', 'ETH-USD'), ('Open', 'ETH-USD'), ('Close', 'ETH-USD'), ('Volume', 'ETH-USD'), ('High', 'ADA-USD'), ('Low', 'ADA-USD'), ('Open', 'ADA-USD'), ('Close', 'ADA-USD'), ('Volume', 'ADA-USD'), ('High', 'BNB-USD'), ('Low', 'BNB-USD'), ('Open', 'BNB-USD'), ('Close', 'BNB-USD'), ('Volume', 'BNB-USD'), ('High', 'XRP-USD'), ('Low', 'XRP-USD'), ('Open', 'XRP-USD'), ('Close', 'XRP-USD'), ('Volume', 'XRP-USD'), ('High', 'SOL-USD'), ('Low', 'SOL-USD'), ('Open', 'SOL-USD'), ('Close', 'SOL-USD'), ('Volume', 'SOL-USD'), ('High', 'DOT-USD'), ('Low', 'DOT-USD'), ('Open', 'DOT-USD'), ('Close', 'DOT-USD'), ('Volume', 'DOT-USD'), ('High', 'DOGE-USD'), ('Low', 'DOGE-USD'), ('Open', 'DOGE-USD'), ('Close', 'DOGE-USD'), ('Volume', 'DOGE-USD'), ('High', 'MATIC-USD'), ('Low', 'MATIC-USD'), ('Open', 'MATIC-U

## 6. Data Quality Check

In [23]:
# Standardize column names to match expected format
if downloaded_data:
    #combined_df.columns = combined_df.columns.str.capitalize()
    print("DATA QUALITY REPORT")
    print("=" * 70)
    
    # Check for missing values
    print("\nMissing values per column:")
    print(combined_df.isnull().sum())
    
    # Check for duplicates
    #duplicates = combined_df.duplicated(subset=['Symbol', 'Date']).sum()
    #print(f"\nDuplicate rows (Symbol + Date): {duplicates}")
    
    # Check data types
    print("\nData types:")
    print(combined_df.dtypes)
    
    # Basic statistics
    print("\nBasic statistics:")
    display(combined_df[['High', 'Low', 'Open', 'Close', 'Volume']].describe())
    
    print("\n" + "=" * 70)

DATA QUALITY REPORT

Missing values per column:
Price   Ticker 
SNo                    0
Name                   0
Symbol                 0
Date                   0
High    BTC-USD    50142
                   ...  
        XTZ-USD    50142
Low     XTZ-USD    50142
Open    XTZ-USD    50142
Close   XTZ-USD    50142
Volume  XTZ-USD    50142
Length: 130, dtype: int64

Data types:
Price   Ticker 
SNo                         int64
Name                       object
Symbol                     object
Date               datetime64[ns]
High    BTC-USD           float64
                        ...      
        XTZ-USD           float64
Low     XTZ-USD           float64
Open    XTZ-USD           float64
Close   XTZ-USD           float64
Volume  XTZ-USD           float64
Length: 130, dtype: object

Basic statistics:


Price            High                                                      \
Ticker        BTC-USD      ETH-USD      ADA-USD      BNB-USD      XRP-USD   
count     2144.000000  2144.000000  2144.000000  2144.000000  2144.000000   
mean     47122.744732  2200.266789     0.664730   382.031378     0.899659   
std      31586.499769  1215.345422     0.561245   256.765959     0.820695   
min       5331.833984   116.021622     0.025993    10.236060     0.146911   
25%      21887.842285  1466.256866     0.314808   237.353790     0.393554   
50%      39919.404297  2073.474243     0.479171   327.730377     0.558504   
75%      64401.476562  3159.720398     0.888294   587.880814     0.964880   
max     126198.070312  4953.732910     3.099186  1370.546021     3.650210   

Price                                                                    ...  \
Ticker      SOL-USD      DOT-USD     DOGE-USD    MATIC-USD     AVAX-USD  ...   
count   2044.000000  1912.000000  2144.000000  1910.000000  1881.000000  ...   
mean      88.333999    11.562919     0.130796     0.741822    31.479961  ...   
std       77.276799    11.124843     0.110602     0.584657    24.512762  ...   
min        0.559759     2.658334     0.001612     0.009060     2.939915  ...   
25%       20.954745     4.711030     0.062808     0.320947    16.332380  ...   
50%       50.816286     6.512243     0.094852     0.702249    23.891144  ...   
75%      155.127533    12.769127     0.187703     1.037802    37.535938  ...   
max      294.334961    55.004974     0.737567     2.923235   146.217865  ...   

Price         Volume                                                          \
Ticker      ALGO-USD       VET-USD       TRX-USD       FIL-USD       ETC-USD   
count   2.144000e+03  2.144000e+03  2.144000e+03  2.144000e+03  2.144000e+03   
mean    1.707493e+08  2.463292e+08  9.648812e+08  3.685104e+08  8.772122e+08   
std     2.718373e+08  4.615803e+08  1.111324e+09  6.563340e+08  1.852413e+09   
min     1.231705e+07  8.857047e+06  9.534694e+07  5.112400e+04  3.792256e+07   
25%     4.841064e+07  4.414110e+07  3.071530e+08  9.657490e+07  1.457415e+08   
50%     8.517357e+07  8.888660e+07  6.720867e+08  1.787903e+08  3.658133e+08   
75%     1.833314e+08  2.264485e+08  1.239387e+09  3.785750e+08  9.378294e+08   
max     4.812070e+09  5.799175e+09  1.397316e+10  1.236798e+10  4.272141e+10   

Price                                                                         
Ticker       XMR-USD      AAVE-USD     THETA-USD       EOS-USD       XTZ-USD  
count   2.144000e+03  1.869000e+03  2.144000e+03  2.144000e+03  2.144000e+03  
mean    2.460846e+08  2.869689e+08  1.068432e+08  1.027731e+09  1.351454e+08  
std     1.003018e+09  2.888567e+08  1.832263e+08  1.664598e+09  2.245491e+08  
min     2.683922e+07  0.000000e+00  7.081570e+05  1.809250e+05  7.785187e+06  
25%     6.792667e+07  1.056716e+08  1.736410e+07  1.009060e+08  2.300487e+07  
50%     9.959165e+07  2.096948e+08  3.307451e+07  2.293125e+08  4.852616e+07  
75%     1.822390e+08  3.721684e+08  1.182177e+08  1.494619e+09  1.577805e+08  
max     2.895912e+10  2.838573e+09  2.582774e+09  2.032871e+10  2.721405e+09  

[8 rows x 125 columns]

## 7. Next Steps

### ✅ Data Downloaded Successfully!

**What you have now:**
- Individual CSV files in `crypto_data_new/` folder
- Combined dataset: `crypto_data_combined.csv`
- Data from 2020 to present day

**Next steps:**
1. Move your old CSV files to `crypto_data_old/` folder (backup)
2. Run `01_data_processing.ipynb` with the new data
3. Run `02_exploratory_data_analysis.ipynb`
4. Run `03_ml_models.ipynb`

**Need to update data later?**
- Just run this notebook again!
- It will download the latest data automatically